In [ ]:
# Import necessary PyTorch libraries
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms


# Additional libraries for visualization and utilities
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def get_device():
    """Selects the best available device for PyTorch computations.

    Returns:
        torch.device: The selected device.
    """

    if torch.cuda.is_available():
        return torch.device('cuda')
    elif torch.backends.mps.is_available():
        return torch.device('mps')
    else:
        return torch.device('cpu')

device = get_device()
print(f"using device: {device}")

In [ ]:
# Import the adapted Echo noise functions
from echo import echo_sample,echo_loss

In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import Compose, ToTensor, Normalize, Grayscale

# Define transformations: Convert to grayscale, resize if needed, and normalize the data
transform = Compose([
    Grayscale(num_output_channels=1),  # Convert to grayscale
    ToTensor(),
    Normalize((0.5,), (0.5,))  # Normalize with single channel
])

# Load the CIFAR-10 dataset
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Splitting dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False)

print("Data loaders created for training and validation.")

In [ ]:
from segmentation_models_pytorch import Unet

class Encoder(nn.Module):
    def __init__(self, input_shape, latent_dims):
        super(Encoder, self).__init__()
        self.input_shape = input_shape
        self.latent_dims = latent_dims

        self.unet = Unet(
            encoder_name="resnet18",
            encoder_depth=3,
            encoder_weights=None,
            decoder_use_batchnorm=True,
            decoder_channels=(latent_dims[2], latent_dims[1], latent_dims[0]),
            decoder_attention_type=None,
            in_channels=input_shape[0],
            classes=latent_dims[-1],
            activation=None,
        )

        # Output layers
        self.out_mean = nn.Conv2d(latent_dims[-1], input_shape[0], kernel_size=1)
        self.out_log_var = nn.Conv2d(latent_dims[-1], input_shape[0], kernel_size=1)

    def forward(self, x):
        x = self.unet(x)
        f_x = torch.tanh(self.out_mean(x))
        log_var = torch.sigmoid(self.out_log_var(x))
        return f_x, log_var

In [ ]:
from segmentation_models_pytorch import Unet

class Encoder(nn.Module):
    def __init__(self, input_shape, latent_dims):
        super(Encoder, self).__init__()
        self.input_shape = input_shape
        self.latent_dims = latent_dims

        self.unet = Unet(
            encoder_name="resnet18",
            encoder_depth=3,
            encoder_weights=None,
            decoder_use_batchnorm=True,
            decoder_channels=(latent_dims[2], latent_dims[1], latent_dims[0]),
            decoder_attention_type=None,
            in_channels=input_shape[0],
            classes=latent_dims[-1],
            activation=None,
        )

        # Output layers
        self.out_mean = nn.Conv2d(latent_dims[-1], input_shape[0], kernel_size=1)
        self.out_log_var = nn.Conv2d(latent_dims[-1], input_shape[0], kernel_size=1)

    def forward(self, x):
        x = self.unet(x)
        f_x = torch.tanh(self.out_mean(x))
        log_var = torch.sigmoid(self.out_log_var(x))
        return f_x, log_var

In [ ]:
def get_timestep_embedding(timesteps, embedding_dim):
    """
    This matches the implementation in Denoising Diffusion Probabilistic Models:
    From Fairseq.
    Build sinusoidal embeddings.
    This matches the implementation in tensor2tensor, but differs slightly
    from the description in Section 3.5 of "Attention Is All You Need".
    """
    assert len(timesteps.shape) == 1

    half_dim = embedding_dim // 2
    emb = math.log(10000) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, dtype=torch.float32) * -emb)
    emb = emb.to(device=timesteps.device)
    emb = timesteps.float()[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if embedding_dim % 2 == 1:  # zero pad
        emb = torch.nn.functional.pad(emb, (0,1,0,0))
    return emb

In [ ]:
class Decoder(nn.Module):
    def __init__(self, latent_dims, output_shape, timestep_dim=128):
        super(Decoder, self).__init__()
        self.latent_dims = latent_dims
        self.output_shape = output_shape
        self.timestep_dim = timestep_dim

        self.conv1 = nn.Conv2d(1, latent_dims[0], kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(latent_dims[0], latent_dims[1], kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(latent_dims[1], latent_dims[2], kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(latent_dims[2], latent_dims[3], kernel_size=3, stride=2, padding=1)
        self.conv5 = nn.Conv2d(latent_dims[3], latent_dims[4], kernel_size=3, stride=1, padding=1)

        self.timestep_mlp = nn.Sequential(
            nn.Linear(timestep_dim, latent_dims[4]),
            nn.ReLU(),
            nn.Linear(latent_dims[4], latent_dims[4]),
        )

        self.deconv1 = nn.ConvTranspose2d(latent_dims[4] * 2, latent_dims[3], kernel_size=3, stride=1, padding=1)
        self.deconv2 = nn.ConvTranspose2d(latent_dims[3], latent_dims[2], kernel_size=4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose2d(latent_dims[2], latent_dims[1], kernel_size=3, stride=1, padding=1)
        self.deconv4 = nn.ConvTranspose2d(latent_dims[1], latent_dims[0], kernel_size=4, stride=2, padding=1)
        self.deconv5 = nn.ConvTranspose2d(latent_dims[0], output_shape[0], kernel_size=3, stride=1, padding=1)

    def forward(self, x, t):
        timestep_emb = get_timestep_embedding(t, self.timestep_dim)
        timestep_emb = self.timestep_mlp(timestep_emb)

        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = torch.relu(self.conv5(x))

        x = torch.cat([x, timestep_emb[:, :, None, None].repeat(1, 1, x.shape[2], x.shape[3])], dim=1)

        x = torch.relu(self.deconv1(x))
        x = torch.relu(self.deconv2(x))
        x = torch.relu(self.deconv3(x))
        x = torch.relu(self.deconv4(x))
        x = torch.sigmoid(self.deconv5(x))

        return x

In [ ]:
class EchoModel(nn.Module):
    def __init__(self, input_shape, latent_dims, output_shape, T=1000, batch_size=100):
        super(EchoModel, self).__init__()
        self.input_shape = input_shape
        self.latent_dims = latent_dims
        self.output_shape = output_shape
        self.T = T
        self.batch_size = batch_size

        self.encoder = Encoder(input_shape, latent_dims)
        self.decoder = Decoder(latent_dims, output_shape)

        # Define the noise schedule
        self.alpha = self.create_noise_schedule(T)

    def create_noise_schedule(self, T):
        alpha = torch.linspace(0.9999, 1e-5, T)
        return alpha

    def forward(self, x):
        #Calculate f_x and S_x
        f_x, sx_matrix = self.encoder(x)
        
        torch.cuda.empty_cache()
        
        #Calculate epsilon in a detached way
        epsilon = echo_sample((f_x, sx_matrix)).detach()

        torch.cuda.empty_cache()

        #Calculate echo output z 
        z = f_x + sx_matrix * epsilon

        del epsilon
        
        torch.cuda.empty_cache()
        
        #sample a timestep t
        t = np.random.randint(0, self.T)
        
        # Retrieve noise scheduler alpha_T
        alpha_t = self.alpha[t]

        # Calculate square root alphas
        sqrt_alpha_t = torch.sqrt(alpha_t)
        sqrt_one_minus_alpha_t = torch.sqrt(1 - alpha_t)
        
        # Perform the weighted sum
        x_t = sqrt_alpha_t * x + sqrt_one_minus_alpha_t * z

        #Calculate the timestep tensor
        t = torch.tensor([t] * x_t.size(0), dtype=torch.long).to(x_t.device)

        # Perform the noise estimation step according to DDPM
        estimated_z = self.decoder(x_t,t)
        torch.cuda.empty_cache()
        return z, estimated_z

In [ ]:
import time  # Importing time to log the duration

def validate(model, val_loader, device):
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():  # Disable gradient computation during validation
        for data, _ in val_loader:
            data = data.to(device)
            z, estimated_z = model(data)
            reconstruction_loss = nn.functional.mse_loss(z, estimated_z)
            total_val_loss += reconstruction_loss.item()  # Accumulate the validation loss
    
    avg_val_loss = total_val_loss / len(val_loader)  # Calculate average loss
    return avg_val_loss

def train(model, optimizer, train_loader, device, num_epochs, accumulation_steps=2, checkpoint_path="checkpoint.pth"):
    model.train()

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        epoch_start_time = time.time()  # Time tracking for the epoch

        print(f"Starting epoch {epoch+1}/{num_epochs}")
        for batch_idx, (data, _) in enumerate(train_loader):
            batch_start_time = time.time()  # Time tracking for the batch
            data = data.to(device)

            # Forward pass
            z, estimated_z = model(data)
            total_loss = nn.functional.mse_loss(z, estimated_z)
            print(f"total loss: {total_loss}")

            # Log before the backward pass
            print(f"\tBatch {batch_idx+1}/{len(train_loader)}, Forward pass done, starting backward pass.")

            # Backward pass
            if not torch.isnan(total_loss).any():
                total_loss.backward()
            else:
                print(f"Warning: NaN detected in total_loss at batch {batch_idx+1}, skipping backward pass.")

            if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(train_loader):
                optimizer.step()  # Only step the optimizer every `accumulation_steps`
                optimizer.zero_grad()  # Reset gradients only after accumulation

            print(f"total loss item: {total_loss.item()}")

            # Safe-guarding against NaN for epoch_loss
            if not torch.isnan(total_loss).any():
                epoch_loss += total_loss.item()
            else:
                print(f"NaN detected, not adding to epoch_loss at batch {batch_idx+1}")
            print(f"Epoch loss: {epoch_loss}")

            # Log after a batch is processed
            print(f"\tBatch {batch_idx+1}/{len(train_loader)} processed in {time.time() - batch_start_time:.2f} seconds.")

        # Average loss after training for an epoch
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}] completed in {time.time() - epoch_start_time:.2f} seconds, Avg Loss: {avg_loss}")

        # Validation phase
        avg_val_loss = validate(model, val_loader, device)
        print(f"Epoch [{epoch+1}/{num_epochs}] validation completed, Avg Validation Loss: {avg_val_loss}")

    return model


In [ ]:
# Define the input shape, latent dimensions, and output shape
input_shape = (1, 32, 32)  # Shape for grayscale CIFAR-10 (1 channel, 32x32 images)
latent_dims = [32, 64, 128, 256, 512]  # Updated latent dimensions
output_shape = (1, 32, 32)  # Shape for grayscale CIFAR-10

# Create an instance of the EchoModel
model = EchoModel(input_shape, latent_dims, output_shape).to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Define the number of epochs and loss weights
num_epochs = 100

# Train the model
trained_model = train(model, optimizer, train_loader, device, num_epochs)